IDMB movie review dataset 

# Import libraries

In [1]:
import glob
import pandas as pd

In [2]:
def make_data_frame(files, sentiment):
    df = pd.DataFrame(columns=['review', 'sentiment'])
    for file in positive_review_train_files:
        with open(file,'r', encoding='utf-8') as infile:
            txt = infile.read()
            df = df.append({'review': txt, 'sentiment': sentiment }, ignore_index=True)
    return df
  
    

Taking review training files

In [3]:
positive_review_train_files=glob.glob("/Users/luo/IDMB/aclImdb/train/pos/*.txt")
len(positive_review_train_files)

12500

In [ ]:
df_pos_train = make_data_frame(positive_review_train_files, 1)


In [ ]:
df_pos_train.head()

In [ ]:
negative_review_train_files =glob.glob("/Users/luo/IDMB/aclImdb/train/neg/*.txt")
len(negative_review_train_files)

In [ ]:
df_neg_train = make_data_frame(negative_review_train_files, 0)


In [ ]:
df_neg_train.head()

In [ ]:
df_train = df_pos_train.append(df_neg_train, ignore_index=True)

In [ ]:
df_train.head()

In [ ]:
df_train.tail()

Taking review testing files

In [ ]:
positive_review_test_files=glob.glob("/Users/luo/IDMB/aclImdb/test/pos/*.txt")
len(positive_review_test_files)

In [ ]:
df_pos_test = make_data_frame(positive_review_test_files, 1)


In [ ]:
df_pos_test.head()

In [ ]:
negative_review_test_files=glob.glob("/Users/luo/IDMB/aclImdb/test/neg/*.txt")
len(negative_review_test_files)

In [ ]:
df_neg_test = make_data_frame(negative_review_test_files, 0)

In [ ]:
df_neg_test.head()

In [ ]:
df_test = df_pos_test.append(df_neg_test, ignore_index=True)

# Preprocessing